# Testing ATWhiteLight

This notebook is meant to be used a functional checkout of the AuxTel illumination system. It includes all functionality but doesn't necessarily show how the system should be used in operation. It was written within the context of the Tucson Teststand and should be modified to be run on the summit.

In [ ]:
import asyncio, time
from lsst.ts import utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.time import Time
from astropy.table import Table
from lsst.ts import salobj

from labjack import ljm

### Connect to CSC

In [ ]:
d = salobj.Domain()
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=d)
await WhiteLightSource.start_task

### Connect to LabJack

In [ ]:
handle = ljm.openS("T4", "ETHERNET", "139.229.170.19") 
names = ["AIN0_NEGATIVE_CH", "AIN0_RANGE", "AIN0_RESOLUTION_INDEX", "AIN0_SETTLING_US",
         "AIN1_NEGATIVE_CH", "AIN1_RANGE", "AIN1_RESOLUTION_INDEX", "AIN1_SETTLING_US",
        "AIN2_NEGATIVE_CH", "AIN2_RANGE", "AIN2_RESOLUTION_INDEX", "AIN2_SETTLING_US"]
aValues = [199, 10.0, 0, 0,
           199, 10.0, 0, 0,
            199, 10.0, 0, 0]
numFrames = len(names)
ljm.eWriteNames(handle, numFrames, names, aValues)

def readPhotodiode():
    loopAmount = 10
    # Read AIN's from the LabJack with eReadNames in a loop.
    names = ["AIN0"]
    numFrames = 1
    intervalHandle = 1
    ljm.startInterval(intervalHandle, 50000)  # Delay between readings (in microseconds)
    Readings= []
    start = time.time()
    for n in range(loopAmount):
        results = ljm.eReadNames(handle, numFrames, names)
        Readings.append(results[0])
        ljm.waitForNextInterval(intervalHandle)
        stamp = time.time()
        delay = stamp - start
        start = stamp
    
    reading = np.mean(Readings)
    if reading > 0.5:
        lightOn = True
    else:
        lightOn = False
    return lightOn

In [ ]:
readPhotodiode()

### Test White Light Source

In [ ]:
await WhiteLightSource.cmd_start.start()

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(WhiteLightSource, state)

In [ ]:
#Start chiller
tmp = await WhiteLightSource.cmd_setChillerTemperature.set_start(temperature=20)
tmp = await WhiteLightSource.cmd_startChiller.set_start()
# Confirm that the chiller has started and running at 20C

### Cycle turning the light on and off

In [ ]:
for i in range(5):
    tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power = 910)
    await asyncio.sleep(20.0)
    if readPhotodiode():
        print(f"Test {i+1} Light came on")
    else:
        print(f"Test {i+1} Light failed to come on")
    await asyncio.sleep(930.0)
    tmp = await WhiteLightSource.cmd_turnLampOff.set_start()
    await asyncio.sleep(930.0)
    
    

### Cycle turning the light on and off while also cycling the CSC

In [ ]:
# Haven't run this yet.
for i in range(5):
    tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power = 910)
    await asyncio.sleep(20.0)
    if readPhotodiode():
        print(f"Test {i+1} Light came on")
    else:
        print(f"Test {i+1} Light failed to come on")
    await asyncio.sleep(930.0)
    tmp = await WhiteLightSource.cmd_turnLampOff.set_start()
    await asyncio.sleep(930.0)
    # Stop chiller
    tmp = await WhiteLightSource.cmd_stopChiller.start()
    state = salobj.State.DISABLED
    tmp = await salobj.set_summary_state(WhiteLightSource, state)
    state = salobj.State.STANDBY
    tmp = await salobj.set_summary_state(WhiteLightSource, state)
    await asyncio.sleep(30.0)
    await WhiteLightSource.cmd_start.start()
    state = salobj.State.ENABLED
    tmp = await salobj.set_summary_state(WhiteLightSource, state)
    await asyncio.sleep(10.0)
    #Start chiller
    tmp = await WhiteLightSource.cmd_setChillerTemperature.set_start(temperature=20)
    tmp = await WhiteLightSource.cmd_startChiller.set_start()
    await asyncio.sleep(30.0)

    
    

In [ ]:
#Turn off the chiller
tmp = await WhiteLightSource.cmd_stopChiller.start()

In [ ]:
ljm.closeAll()

In [ ]:
ljm.close(handle)